In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import os
import sqlalchemy as sa
import pyodbc
import warnings
import urllib
warnings.filterwarnings('ignore')


In [2]:

path = "../2. Code/Classes"
os.chdir(path)

from Preprocessing_Application_N import DataPreprocessor
from Credentials_Loader import CredentialLoader


loader = CredentialLoader()  # Initializes the loader with default paths
credentials = loader.load_credentials()  # Loads the credentials from .env

username = credentials['username']
password = credentials['password']
server = credentials['server']
database = credentials['reporting_db']

relative_path_to_excel = "../../1. Data/Invoice Not Paid - Manual Finance Input/InvNotPaid 2024-05-31.xlsx"

df = pd.read_excel(relative_path_to_excel, sheet_name="Aggregerad ", engine='openpyxl')



In [3]:
os.getcwd()

'/Users/andreasnilsson/Library/CloudStorage/OneDrive-Nstart/Skrivbordet/Repository Homes/Behavior_Scorecard-DATA-223/2. Code/Classes'

In [4]:
df

,Kontonummer,Betalt belopp,Obetalt belopp,Aviserad obetlad amortering,InvNotPaid
0,5000187,4021.39,18115.61,2788.45,15327.16
1,5000229,0.00,30635.00,5923.80,24711.20
2,5000294,7406.25,22960.75,2134.69,20826.06
3,5000369,377.04,4947.96,368.83,4579.13
4,5000377,0.00,20445.00,1309.76,19135.24
...,...,...,...,...,...
1843,9049859,0.00,1199.00,365.86,833.14
1844,9051194,0.00,462.00,140.49,321.51
1845,9052077,0.00,605.00,184.39,420.61
1846,9052572,0.00,404.00,122.93,281.07


In [5]:
import pandas as pd
import datetime

ToAmco = pd.DataFrame()

# Get today's date
today = datetime.date.today()

# Calculate the first day of the current month
first_day_of_current_month = datetime.date(today.year, today.month, 1)

# Subtract one day from the first day of the current month to get the last day of the previous month
last_day_of_previous_month = first_day_of_current_month - datetime.timedelta(days=1)

# Assuming ToAmco is your DataFrame

ToAmco['AccountNumber'] = df.Kontonummer
ToAmco['amount'] = df.InvNotPaid
ToAmco['SnapshotDate'] = last_day_of_previous_month

# Adjust order before writing to DB
writeDf =pd.DataFrame()
writeDf = ToAmco[['SnapshotDate','AccountNumber','amount']]
writeDf['AccountNumber'] = writeDf['AccountNumber'].astype(str)


In [6]:
writeDf

,SnapshotDate,AccountNumber,amount
0,2024-05-31,5000187,15327.16
1,2024-05-31,5000229,24711.20
2,2024-05-31,5000294,20826.06
3,2024-05-31,5000369,4579.13
4,2024-05-31,5000377,19135.24
...,...,...,...
1843,2024-05-31,9049859,833.14
1844,2024-05-31,9051194,321.51
1845,2024-05-31,9052077,420.61
1846,2024-05-31,9052572,281.07


In [7]:

# Update the driver to 'ODBC Driver 17 for SQL Server' for Native Client 17
# This assumes you're using ODBC Driver 17, which is the usual driver used with SQL Server Native Client 17 installations
engine = sa.create_engine(f'mssql+pyodbc://{username}:{password}@{server}:1433/{database}?driver=ODBC+Driver+17+for+SQL+Server')


In [8]:
# Upload DataFrame to SQL table
writeDf.to_sql('InvoicedNotPaid', con=engine, index=False, if_exists='append', schema='nystart')

450